In [3]:

# import library
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time


agent = {"User-Agent":'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
wines = {}

# url = 'https://www.wine-searcher.com/find/riesling/2021'
# req = requests.get(url, headers=agent)
# content = req.text




# DRIVER_PATH = r'/Users/magnusgreve/Downloads/chromedriver'
# options = Options()
# options.add_argument("--user-data-dir=chrome-data") 
# options.headless = False
# options.add_argument("--window-size=1920,1200")
# driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
# time.sleep(5)


for i in range(2017,2022):
    response = requests.get(f"https://www.wine-searcher.com/find/bordeax/{i}/norway", headers=agent)
    if not response.ok:
        print(response.status_code)
    else:
        soup = BeautifulSoup(response.text)
        print('Finding wines')
        cards = soup.findAll(class_='card card-product')
        print('Looping through wines')
        for i in cards:
            name = i.find(class_='card-product__name')
            rating = i.find(class_='badge-rating badge badge-pill')
            try:
                wines[name.text.replace("\n", "")] = rating.text.split('/')[0]
            except:
                wines[name.text.replace("\n", "")] = 0
        print(wines)
        time.sleep(5)

df = pd.DataFrame({'Wines': list(wines.keys()), 'Points': list(wines.values())})


Finding wines
Looping through wines
{'2017 Chateau Mouton Rothschild': '95 ', '2017 Petrus': '96 ', '2017 Chateau Haut-Brion': '95 ', '2017 Chateau Lynch-Bages': '93 ', '2017 Chateau Cheval Blanc': '95 ', '2017 Chateau Pichon Longueville Comtesse de Lalande': '95 ', "2017 Chateau Leoville-Las Cases 'Grand Vin de Leoville'": '95 ', '2017 Chateau Montrose': '94 ', '2017 Chateau Palmer': '95 ', '2017 Chateau Ducru-Beaucaillou': '95 ', '2017 Chateau Leoville Poyferre': '94 ', '2017 Chateau Pichon-Longueville au Baron de Pichon-Longueville': '94 ', '2017 Chateau La Mission Haut-Brion': '95 ', '2017 Chateau Leoville Barton': '94 ', '2017 Chateau Figeac': '95 ', '2017 Chateau Talbot': '91 ', '2017 Chateau Calon-Segur': '94 ', '2017 Chateau Lafleur': '97 ', '2017 Chateau Ausone': '95 ', '2017 Vieux Chateau Certan': '96 ', '2017 Chateau Canon': '94 ', '2017 Chateau Gruaud-Larose': '92 ', '2017 Chateau Rauzan-Segla': '93 ', '2017 Chateau Giscours': '92 ', '2017 Chateau Beychevelle': '93 ', '2017

In [4]:

df.to_csv('2017.csv')

In [6]:
df1 = pd.read_csv('databordeaux.csv')
df2 = pd.read_csv('2017.csv')
df3 = pd.read_csv('2010.csv')

In [9]:
df1 = df1[['Wines', 'Points']]
df2 = df2[['Wines', 'Points']]
df3 = df3[['Wines', 'Points']]

/var/folders/b4/5w_zxjl95ql23z3g4v334ztc0000gn/T/ipykernel_6931/2407101422.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df3)


,Wines,Points
0,2000 Chateau Mouton Rothschild,94
1,2000 Chateau Lafite Rothschild,96
2,2000 Petrus,97
3,2000 Chateau Margaux,97
4,2000 Chateau Latour,96
...,...,...
320,2017 Chateau Pichon Longueville Comtesse de La...,91
321,2017 Chateau Chasse-Spleen,88
322,2017 Chateau Leoville-Las Cases 'Clos du Marquis',92
323,2017 Le Clarence de Haut-Brion - Chateau Bahan...,91


/var/folders/b4/5w_zxjl95ql23z3g4v334ztc0000gn/T/ipykernel_6931/948459739.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)


In [19]:
df.to_csv('fullbordeauxdata.csv',index=False)